In [46]:
from langgraph.graph import START, END, StateGraph
from typing import TypedDict,Annotated

In [47]:
class CricketBatsmanState(TypedDict):
    runs: int
    balls: int
    fours: int
    sixes: int
    sr: Annotated[float, "merge"]
    bpb: Annotated[float, "merge"]
    boundary_percentage: Annotated[float, "merge"]
    summary: Annotated[str, "merge"]

In [48]:
def calculate_sr(state: CricketBatsmanState) -> dict:
    sr = (state['runs'] / state['balls']) * 100
    return {'sr': sr}

In [49]:
def calculate_bpb(state: CricketBatsmanState) -> dict:
    bpb = state['balls'] / (state['fours'] + state['sixes'])
    return {'bpb': bpb}

In [50]:
def calculate_boundary_percentage(state: CricketBatsmanState) -> dict:
    boundary_percentage = (((state['fours'] * 4) + (state['sixes'] * 6)) / state['runs']) * 100
    return {'boundary_percentage': boundary_percentage}

In [51]:
def summary(state: CricketBatsmanState) -> dict:
    summary = (
        f"Runs: {state['runs']}, Balls: {state['balls']}, "
        f"Fours: {state['fours']}, Sixes: {state['sixes']}, "
        f"Strike Rate: {state['sr']:.2f}, BPB: {state['bpb']:.2f}, "
        f"Boundary%: {state['boundary_percentage']:.2f}"
    )
    return {'summary': summary}

In [52]:
graph = StateGraph(CricketBatsmanState)

graph.add_node('calculate_sr', calculate_sr)
graph.add_node('calculate_bpb', calculate_bpb)
graph.add_node('calculate_boundary_percentage', calculate_boundary_percentage)
graph.add_node('summary', summary)


graph.add_edge(START, 'calculate_sr')
graph.add_edge(START, 'calculate_bpb')
graph.add_edge(START, 'calculate_boundary_percentage')
graph.add_edge('calculate_sr', 'summary')
graph.add_edge('calculate_bpb', 'summary')
graph.add_edge('calculate_boundary_percentage', 'summary')

graph.add_edge('summary', END)

workflow = graph.compile()

In [64]:
initial_state = {
    'runs': 100,
    'balls': 40,
    'fours': 10,
    'sixes': 5
}

final_state = workflow.invoke(initial_state)

print("Strike Rate : ",final_state['sr'])
print("boundary Per ball : ",final_state['bpb'])
print("Boundary Percentage : ",final_state['boundary_percentage'])
print("\nComplete Summary : ",final_state['summary'])

Strike Rate :  250.0
boundary Per ball :  2.6666666666666665
Boundary Percentage :  70.0

Complete Summary :  Runs: 100, Balls: 40, Fours: 10, Sixes: 5, Strike Rate: 250.00, BPB: 2.67, Boundary%: 70.00
